## !!  Important  !!
**Before** running this notebook make sure all of the education related datasets are properly downloaded / scraped and saved into the landing folder.

Also please make sure that the **Demographics** preprocessing notebook is run **before** you run this one
- for datasets that are scraped check that they are in the landing folder 
- for datasets that need to be manually downloaded BEFORE running this notebook:

      - please move the csv file (university_locations.csv) to the data/landing folder

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
from fuzzywuzzy import process, fuzz
import os


this function checks for missing values and Nan

In [ ]:
def check_for_nans(df, stage):
    print(f"\nChecking for NaN values after {stage}:\n")
    # check for any missing values in each column
    missing_values = df.isnull().sum()
    # show columns with missing values
    print("Columns with NaN values and their counts:")
    print(missing_values[missing_values > 0])
    # show rows with any NaN values
    rows_with_missing = df[df.isnull().any(axis=1)]
    print(f"Rows with NaN values ({len(rows_with_missing)} rows):")
    print(rows_with_missing)
    print("-" * 50)

### Preprocessing for TAFE locations

In [4]:
# reading TAFE dataset
tafe_df = pd.read_csv('../data/landing/TAFE_locations.csv')

In [5]:
# extract suburbs where TAFE campuses are located
def extract_suburbs(location):
    suburbs = []
    # split by semicolumn and space to get each part of the multiple campus locations
    for part in location.split('; '):
        # for each part get everything before the first colon (the suburb)
        suburb = part.split(':')[0]
        suburbs.append(suburb)
    return suburbs

# apply function to TAFE df
tafe_df['suburbs'] = tafe_df['Locations'].apply(extract_suburbs)
# each suburb gets an individual row 
tafe_df = tafe_df.explode('suburbs')

In [6]:
# drop locations column - dont need anymore
tafe_df = tafe_df.drop(columns='Locations')
# change column name to align format
tafe_df = tafe_df.rename(columns={'Institute Name': 'institute_name'})
tafe_df = tafe_df.rename(columns={'suburbs': 'suburb'})
tafe_df['institute_name'] = tafe_df['institute_name'].str.lower()
tafe_df['suburb'] = tafe_df['suburb'].str.lower()

In [395]:
tafe_df

,institute_name,suburb
0,bendigo tafe,bendigo
0,bendigo tafe,bendigo
0,bendigo tafe,castlemaine
0,bendigo tafe,echuca
1,box hill institute,box hill
...,...,...
14,victoria university,sunshine
14,victoria university,werribee
15,william angliss institute,melbourne
16,wodonga tafe,barnawartha north


### Preprocessing for University locations

In [7]:
# reading university dataset
university_df = pd.read_csv('../data/landing/university_locations.csv')

In [8]:
# drop columns that we dont need 
university_df = university_df.drop(columns=['Campus', 'City or Town', 'Post Code', 'Name'])
# rename columns 
university_df = university_df.rename(columns={'University': 'institute_name'})
university_df['institute_name'] = university_df['institute_name'].str.lower()


In [9]:
# extract the suburb information from address column
pattern = r'(?:,\s*(.*?)\s*V)|(?:\s+([^\s]+)\s*V)' # i want the text between , and VIC, if theres no , then between a space and VIC
university_df['suburb'] = university_df['Address'].str.extract(pattern).bfill(axis=1).iloc[:, 0]
university_df['suburb'] = university_df['suburb'].str.lower()

# drop columnds that we dont need
university_df = university_df.drop(columns=['Address'])

# remove any punctuation
university_df['suburb'] = university_df['suburb'].str.replace('[,"\']', '', regex=True)

##### Combine into Tertiary Insitution location csv

In [10]:
tertiary_df = pd.concat([tafe_df, university_df], ignore_index=True)

# save as csv to data/raw folder 
tertiary_df.to_csv('../data/raw/tertiary_insitutions_suburbs.csv', index=False)

### Preprocessing for Primary Secondary location

In [11]:
# reading dataset
prim_sec_df = pd.read_csv('../data/landing/2023_primary_secondary_locations.csv', encoding='ISO-8859-1')

In [12]:
# list of columns to keep
columns_to_keep = ['School_Name', 'School_Type', 'Address_Town', 'Address_Postcode', 'X', 'Y']
# keep selected columns
prim_sec_df = prim_sec_df[columns_to_keep]
# change column names
prim_sec_df = prim_sec_df.rename(columns={
    'School_Name': 'school_name',
    'School_Type': 'school_type',
    'Address_Town': 'suburb',
    'Address_Postcode': 'postcode',
    'X': 'longitude',
    'Y': 'latitude'
})

prim_sec_df['school_name'] = prim_sec_df['school_name'].str.lower()
prim_sec_df['suburb'] = prim_sec_df['suburb'].str.lower()
prim_sec_df['school_type'] = prim_sec_df['school_type'].str.lower()

In [13]:
# split df into primary schools and secondary schools
primary_schools_df = prim_sec_df[prim_sec_df['school_type'].isin(['primary', 'pri/sec'])]
secondary_schools_df = prim_sec_df[prim_sec_df['school_type'].isin(['secondary', 'pri/sec'])]

creating ranking for primary schools

In [104]:
all_enrollments = pd.read_csv('../data/landing/All_schools_enrollments.csv', encoding='ISO-8859-1')

In [105]:
# extract required columns
prim_enrollments = all_enrollments[['School_Name', 'School_Type', '"Primary Total"']]
# make column names and values to lowercase
prim_enrollments.columns = [col.lower().replace('"', '').strip() for col in prim_enrollments.columns]
prim_enrollments = prim_enrollments.apply(lambda col: col.str.lower() if col.dtype == 'object' else col)
# filter to only include primary schools
prim_enrollments = prim_enrollments[prim_enrollments['school_type'].isin(['primary', 'pri/sec'])]
# rename 'primary total' to 'total_enrollments'
prim_enrollments = prim_enrollments.rename(columns={'primary total': 'total_enrollments'})
# drop 'school_type' column
prim_enrollments = prim_enrollments.drop(columns=['school_type'])

                  school_name  total_enrollments
4         st patrick's school              247.0
5   st alipius' parish school              238.0
8            st mary's school              163.2
9         sacred heart school              307.0
10           st mary's school              286.6


In [124]:
# dealing with duplicates: keeping entries with the highest total_enrollments for each school_name
prim_enrollments = prim_enrollments.sort_values('total_enrollments', ascending=False).drop_duplicates(subset='school_name', keep='first')
# round enrollment counts up to nearest whole number
prim_enrollments['total_enrollments'] = np.ceil(prim_enrollments['total_enrollments'])

merging locations with enrollment number

In [127]:
# conduct exact match based on 'school_name'
primary_schools_exact_matched = pd.merge(primary_schools_df, 
                                    prim_enrollments[['school_name', 'total_enrollments']], 
                                    on='school_name', how='left')
# identify schools without exact matches (where total_enrollments is NaN)
unmatched_schools = primary_schools_exact_matched[primary_schools_exact_matched['total_enrollments'].isna()]['school_name'].tolist()

# fuzzy match function for the identified unmatched schools
def fuzzy_match(school_name, choices, threshold=90):
    match, score = process.extractOne(school_name, choices)
    if score >= threshold:
        return match
    else:
        return None
# get list of school names from prim_enrollments for matching
prim_enrollment_names = prim_enrollments['school_name'].tolist()
# dictionary to store fuzzy matches
fuzzy_matches = {}
# apply fuzzy matching
for school in unmatched_schools:
    match = fuzzy_match(school, prim_enrollment_names)
    if match:
        fuzzy_matches[school] = match
# update original df with fuzzy matched results
for school, match in fuzzy_matches.items():
    enrollment_value = prim_enrollments[prim_enrollments['school_name'] == match]['total_enrollments'].values[0]
    primary_schools_exact_matched.loc[primary_schools_exact_matched['school_name'] == school, 'total_enrollments'] = enrollment_value
# remove rows for remaining unmatched schools
primary_schools_df = primary_schools_exact_matched.dropna(subset=['total_enrollments'])

merging locations data (which now includes enrollment number) to demographics for population

In [240]:
# reading demographics data
demographics = pd.read_csv('../data/curated/demographics.csv')
pop_density_df = demographics[['sa2_name', 'area_km2', 'pop_density_persons_km2']]
pop_density_df = pop_density_df.rename(columns={'sa2_name': 'suburb'})

In [327]:
# replace entries of 0 with 0.00001 in 'pop_density_persons_km2'
pop_density_df.loc[pop_density_df['pop_density_persons_km2'] == 0, 'pop_density_persons_km2'] = 0.00001
# create new column 'population' by multiplying 'area_km2' and 'pop_density_persons_km2'
pop_density_df['population'] = pop_density_df['area_km2'] * pop_density_df['pop_density_persons_km2']

In [328]:
# merge df on 'suburb' to add the population data
primary_schools_df_with_population = primary_schools_df.merge(
    pop_density_df[['suburb', 'population']], 
    on='suburb', 
    how='left'
)
# fill illing NaN values in the 'population' column with 0 for unmatched suburbs
primary_schools_df_with_population['population'] = primary_schools_df_with_population['population'].fillna(0)

it seems as though the demographic data our team has acquired does not include all of the suburbs that the education dataset includes. for the suburbs that do not have a recorded population, fill in estimated population based on the median population of suburbs with similar enrolment counts

In [329]:
# loop through each row where population is 0
for index, row in primary_schools_df_with_population[primary_schools_df_with_population['population'] == 0].iterrows():
    # get the total_enrollments value for the current row
    enrollments = row['total_enrollments']
    
    # find rows with non-zero population and total_enrollments within a difference of 16
    similar_rows = primary_schools_df_with_population[
        (primary_schools_df_with_population['population'] != 0) &
        (primary_schools_df_with_population['total_enrollments'].between(enrollments - 16, enrollments + 16))
    ].copy()
    
    # if similar rows are found, select the two closest entries by total_enrollments
    if not similar_rows.empty:
        similar_rows['enrollment_diff'] = (similar_rows['total_enrollments'] - enrollments).abs()
        
        # calculate median population of the closest entries
        median_population = similar_rows['population'].median()
        # update the population value for the current row
        primary_schools_df_with_population.at[index, 'population'] = median_population

calculate enrolment percentage and rank

In [331]:
# calculate enrollment_percentage as total_enrollments/population
primary_schools_df_with_population['enrollment_percentage'] = (
    primary_schools_df_with_population['total_enrollments'] / primary_schools_df_with_population['population']
)

# handle any potential division by zero or NaN values by creating a new series
primary_schools_df_with_population['enrollment_percentage'] = primary_schools_df_with_population['enrollment_percentage'].fillna(0)
primary_schools_df_with_population['enrollment_percentage'] = primary_schools_df_with_population['enrollment_percentage'].replace([float('inf'), -float('inf')], 0)

# create rank column based on the enrollment_percentage
primary_schools_df_with_population['rank'] = primary_schools_df_with_population['enrollment_percentage'].rank(
    method='min', ascending=False
).astype(int)

In [332]:
primary_schools_df_final = primary_schools_df_with_population
primary_schools_df_final.to_csv('../data/raw/primary_school_locations.csv', index=False)

### Preprocessing for Highschool rankings

##### Creating highschool rankings using school completion and achievement information

In [356]:
# reading dataset
hs_achievement = pd.read_csv('../data/landing/2023SeniorSecondaryCompletionAndAchievementInformation.csv')

In [358]:
# removing trailing and leading spaces from column names
hs_achievement.columns = hs_achievement.columns.str.strip()
# remove trailing and leading spaces from string entries in df
hs_achievement = hs_achievement.apply(lambda col: col.map(lambda x: x.strip() if isinstance(x, str) else x))

columns_to_keep = ['School', 
                   'Number of VCE and VCE Vocational Major (VM) studies at Units 3 and 4 level with enrolments', 
                   'Percentage of satisfactory VCE completions', 
                   'Median VCE study score', 
                   'Percentage of study scores of 40 and over']
hs_achievement = hs_achievement[columns_to_keep]

In [359]:
cols_to_convert = hs_achievement.columns.difference(['School'])
# convert the non-school columns to numeric, coercing errors to NaN
hs_achievement[cols_to_convert] = hs_achievement[cols_to_convert].apply(pd.to_numeric, errors='coerce')

# fill missing values with the median of their respective columns
for col in cols_to_convert:
    median_value = hs_achievement[col].median()
    # assign column with filled NaN values directly back to df
    hs_achievement[col] = hs_achievement[col].fillna(median_value)

In [361]:
# define weights for each column based on importance
weights = {
    'Percentage of study scores of 40 and over': 0.4,
    'Median VCE study score': 0.3,
    'Number of VCE and VCE Vocational Major (VM) studies at Units 3 and 4 level with enrolments': 0.15,
    'Percentage of satisfactory VCE completions': 0.15
}

# normalise scores by dividing each by its maximum value, so they are on the same scale
for col in weights.keys():
    hs_achievement[col] = hs_achievement[col] / hs_achievement[col].max()

# calculate total score as the weighted sum of the columns
hs_achievement['total_score'] = (
    hs_achievement['Percentage of study scores of 40 and over'] * weights['Percentage of study scores of 40 and over'] +
    hs_achievement['Median VCE study score'] * weights['Median VCE study score'] +
    hs_achievement['Number of VCE and VCE Vocational Major (VM) studies at Units 3 and 4 level with enrolments'] * weights['Number of VCE and VCE Vocational Major (VM) studies at Units 3 and 4 level with enrolments'] +
    hs_achievement['Percentage of satisfactory VCE completions'] * weights['Percentage of satisfactory VCE completions']
)

# rank schools based on the total score
hs_achievement['school_rank'] = hs_achievement['total_score'].rank(ascending=False)

In [363]:
# make df more organised
hs_achievement = hs_achievement.sort_values(by='school_rank')
hs_achievement['school_rank'] = hs_achievement['school_rank'].round().astype(int)
cols = hs_achievement.columns.tolist()
# move 'school_rank' to the second position
cols.insert(1, cols.pop(cols.index('school_rank')))
# reorder df
hs_achievement = hs_achievement[cols]
# rename the columns: lowercase and replace spaces with underscores
hs_achievement.columns = hs_achievement.columns.str.lower().str.replace(' ', '_')
# convert all entries in the 'school' column to lowercase
hs_achievement['school'] = hs_achievement['school'].str.lower()
# rename columns to align all datasets 
hs_achievement = hs_achievement.rename(columns={
    'school_rank': 'rank',
    'school': 'school_name',
})

##### fuzzy match school_names to map rankings to schools

In [367]:
# function to remove common words for matching purposes
def clean_school_name(name):
    common_words = ['college', 'school', 'centre', 'senior', 'prep', 'year', 'secondary', 
                    'primary', 'grammar', 'christian', 'catholic', 'p-9', 'p-12', '-9', 'sec']
    tokens = name.split()
    filtered_tokens = [token for token in tokens if token.lower() not in common_words]
    return ' '.join(filtered_tokens)

# fuzzy match function
def fuzzy_match_school(school, achievement_schools_cleaned):
    return process.extractOne(school, achievement_schools_cleaned)

# clean school name list for matching
achievement_schools_cleaned = [clean_school_name(school) for school in hs_achievement['school_name'].tolist()]
secondary_schools_cleaned = [clean_school_name(school) for school in secondary_schools_df['school_name'].tolist()]

# conduct fuzzy matching using cleaned school names
best_matches = []
match_scores = []

for school in secondary_schools_cleaned:
    match, score = fuzzy_match_school(school, achievement_schools_cleaned)
    best_matches.append(match)
    match_scores.append(score)  

# create df to store matches and scores
match_df = pd.DataFrame({
    'original_school_name_in_secondary': secondary_schools_df['school_name'],  # original school names
    'cleaned_school_name_in_secondary': secondary_schools_cleaned,  # cleaned school names
    'best_match_cleaned': best_matches,  # best match
    'match_score': match_scores  # score
})

# set match score threshold
threshold = 88
# filter matches exceeding threshold
match_df_filtered = match_df[match_df['match_score'] >= threshold]

# prep hs_achievement_cleaned df
hs_achievement_cleaned = pd.DataFrame({
    'original_school_name_in_achievement': hs_achievement['school_name'],  # Original names
    'cleaned_school_name_in_achievement': achievement_schools_cleaned,  # Cleaned names
    'rank': hs_achievement['rank']  # Rank
})

# merge the cleaned school name matches to get the rank
match_df_filtered = match_df_filtered.merge(
    hs_achievement_cleaned[['cleaned_school_name_in_achievement', 'rank']], 
    left_on='best_match_cleaned', right_on='cleaned_school_name_in_achievement', how='left'
)

# merge with the secondary_schools_df to include the rank
final_secondary_schools_df = pd.merge(
    secondary_schools_df, 
    match_df_filtered[['original_school_name_in_secondary', 'rank']], 
    left_on='school_name', 
    right_on='original_school_name_in_secondary', 
    how='left'
)

final_secondary_schools_df.drop(columns=['original_school_name_in_secondary'], inplace=True)

troubleshooting function for ranks

In [370]:
# identify entries with NaN rank
nan_rank_entries = final_secondary_schools_df[final_secondary_schools_df['rank'].isnull()].copy()

# function to identify why rank is NaN for each entry
def find_nan_reason(row, match_df, hs_achievement_cleaned, threshold):
    school_name = row['school_name']
    # check if it exists in match_df
    match_entry = match_df[match_df['original_school_name_in_secondary'] == school_name]
    if match_entry.empty:
        return "No match found in match_df"
    
    # check if match score is below threshold
    match_score = match_entry['match_score'].values[0]
    if match_score < threshold:
        return f"Match score ({match_score}) below threshold"
    
    # check if the best match cleaned name exists in hs_achievement_cleaned
    best_match = match_entry['best_match_cleaned'].values[0]
    if hs_achievement_cleaned[hs_achievement_cleaned['cleaned_school_name_in_achievement'] == best_match].empty:
        return "Best match cleaned name not found in hs_achievement_cleaned"
    
    # if all else passes = unexpected NaN
    return "Unknown reason"

# apply function to find the reason for NaN rank
nan_rank_entries['reason_for_nan'] = nan_rank_entries.apply(
    find_nan_reason, axis=1, 
    match_df=match_df, 
    hs_achievement_cleaned=hs_achievement_cleaned, 
    threshold=threshold
)

In [371]:
# display the resulting df with NaN rank entries and reasons
nan_rank_entries[['school_name', 'rank', 'reason_for_nan']]

,school_name,rank,reason_for_nan
1,lake bolac college,NaN,Match score (72) below threshold
8,baden powell p-9 college,NaN,Match score (56) below threshold
16,timbarra p-9 college,NaN,Match score (68) below threshold
17,carranballac p-9 college,NaN,Match score (68) below threshold
18,truganina p-9 college,NaN,Match score (60) below threshold
19,yuille park p-8 community college,NaN,Match score (86) below threshold
20,alamanda k-9 college,NaN,Match score (86) below threshold
27,manangatang p-12 college,NaN,Match score (70) below threshold
32,werrimull p-12 school,NaN,Match score (60) below threshold
37,victorian school of languages,NaN,Match score (77) below threshold


In [374]:
final_secondary_schools_df = final_secondary_schools_df[~final_secondary_schools_df['school_name'].isin(nan_rank_entries['school_name'])]
final_secondary_schools_df = final_secondary_schools_df.dropna(subset=['longitude', 'latitude'])

In [385]:
# sort df by 'school_name' and 'rank'
final_secondary_schools_df = final_secondary_schools_df.sort_values(by=['school_name', 'rank'])
# drop duplicates based on 'school_name'
# keep row with the highest rank 
final_secondary_schools_df = final_secondary_schools_df.drop_duplicates(subset='school_name', keep='first')
final_secondary_schools_df['rank'] = final_secondary_schools_df['rank'].astype(int)

In [386]:
# save df to data/raw
final_secondary_schools_df.to_csv('../data/raw/secondary_school_locations.csv', index=False)

### Create features for final dataset

In [396]:
# protocol final education df without rank (was not used)

# # count unique primary schools by suburb
# primary_counts = primary_schools_df_final['suburb'].value_counts().reset_index()
# primary_counts.columns = ['suburb', 'primary_school_count']

# # count unique secondary schools by suburb
# secondary_counts = final_secondary_schools_df['suburb'].value_counts().reset_index()
# secondary_counts.columns = ['suburb', 'secondary_school_count']

# # count unique tertiary institutions by suburb
# tertiary_counts = tertiary_df['suburb'].value_counts().reset_index()
# tertiary_counts.columns = ['suburb', 'tertiary_institutions_count']

# # merge counts into one df
# final_suburb_counts = primary_counts.merge(secondary_counts, on='suburb', how='outer')
# final_suburb_counts = final_suburb_counts.merge(tertiary_counts, on='suburb', how='outer')

# # fill NaN values with 0 (if any suburbs are missing in some counts)
# final_suburb_counts.fillna(0, inplace=True)

# # convert counts to integers
# final_suburb_counts['primary_school_count'] = final_suburb_counts['primary_school_count'].astype(int)
# final_suburb_counts['secondary_school_count'] = final_suburb_counts['secondary_school_count'].astype(int)
# final_suburb_counts['tertiary_institutions_count'] = final_suburb_counts['tertiary_institutions_count'].astype(int)

# # calculate total education count
# final_suburb_counts['total_education_count'] = (
#     final_suburb_counts['primary_school_count'] +
#     final_suburb_counts['secondary_school_count'] +
#     final_suburb_counts['tertiary_institutions_count']
# )

final df code that includes rank

In [402]:
# count unique primary schools by suburb
primary_counts = primary_schools_df_final['suburb'].value_counts().reset_index()
primary_counts.columns = ['suburb', 'primary_school_count']

# calculate average primary school rank by suburb
primary_avg_rank = primary_schools_df_final.groupby('suburb')['rank'].mean().reset_index()
primary_avg_rank.columns = ['suburb', 'avg_primary_school_rank']

# count unique secondary schools by suburb
secondary_counts = final_secondary_schools_df['suburb'].value_counts().reset_index()
secondary_counts.columns = ['suburb', 'secondary_school_count']

# calculate average secondary school rank by suburb
secondary_avg_rank = final_secondary_schools_df.groupby('suburb')['rank'].mean().reset_index()
secondary_avg_rank.columns = ['suburb', 'avg_secondary_school_rank']

# count unique tertiary institutions by suburb
tertiary_counts = tertiary_df['suburb'].value_counts().reset_index()
tertiary_counts.columns = ['suburb', 'tertiary_institutions_count']

# merge counts into one df
final_suburb_counts = primary_counts.merge(secondary_counts, on='suburb', how='outer')
final_suburb_counts = final_suburb_counts.merge(tertiary_counts, on='suburb', how='outer')

# merge average ranks into the final df
final_suburb_counts = final_suburb_counts.merge(primary_avg_rank, on='suburb', how='left')
final_suburb_counts = final_suburb_counts.merge(secondary_avg_rank, on='suburb', how='left')

# fill NaN values with 0 for count columns and average ranks
final_suburb_counts.fillna({
    'primary_school_count': 0,
    'secondary_school_count': 0,
    'tertiary_institutions_count': 0,
    'avg_primary_school_rank': 0,
    'avg_secondary_school_rank': 0
}, inplace=True)

# convert count columns to integers
final_suburb_counts['primary_school_count'] = final_suburb_counts['primary_school_count'].astype(int)
final_suburb_counts['secondary_school_count'] = final_suburb_counts['secondary_school_count'].astype(int)
final_suburb_counts['tertiary_institutions_count'] = final_suburb_counts['tertiary_institutions_count'].astype(int)

# create binary indicators for presence of primary and secondary schools
final_suburb_counts['has_primary_school'] = (final_suburb_counts['primary_school_count'] > 0).astype(int)
final_suburb_counts['has_secondary_school'] = (final_suburb_counts['secondary_school_count'] > 0).astype(int)

# calculate total education count
final_suburb_counts['total_education_count'] = (
    final_suburb_counts['primary_school_count'] +
    final_suburb_counts['secondary_school_count'] +
    final_suburb_counts['tertiary_institutions_count']
)

         suburb  primary_school_count  secondary_school_count  \
0    abbotsford                     2                       0   
1    aberfeldie                     1                       1   
2       aintree                     1                       0   
3  aireys inlet                     1                       0   
4         airly                     1                       0   

   tertiary_institutions_count  avg_primary_school_rank  \
0                            0                    964.5   
1                            0                    643.0   
2                            0                    215.0   
3                            0                   1457.0   
4                            0                   1626.0   

   avg_secondary_school_rank  has_primary_school  has_secondary_school  \
0                        0.0                   1                     0   
1                      172.0                   1                     1   
2                        0.0    

In [406]:
# save df to data/curated 
final_suburb_counts.to_csv('../data/curated/education_df.csv', index=False)